## Proyecto EDA - Accidentalidad de Madrid

Este EDA va sobre la accidental de la cuidad de Madrid en los años 2020, 2021, 2022, 2023, 2024 y 2025. Los Dataset han diso descargados de la web....

- Librerias importantes que necesitamos en el EDA:

In [4]:
import pandas as pd
import os
import glob
import numpy as np
pd.options.mode.copy_on_write=True


- Primero vamos hacer una exploracion de las varibales que tenemos.

- Explorando el Dataset nos damos cuenta que hay varios numeros de expedientes iguales y la conclusion es que hay un num expediente por persona implicada en el accidente y es importante entender esto porque puede afectar al calculo de numero de accidentes, ademas que si hay varias personas implicadas en el acidente puede ser que haya personas que salgan leves, graves,etc

Objetivos del EDA:

1. EDA Temporal
Columnas útiles: fecha, hora
Qué puedes analizar
- Distribución de accidentes por día, mes, día de la semana
- Accidentes por franja horaria
- Identificar horas pico
- Accidentes en madrugada vs tarde vs noche
- Evolución diaria (si tienes varios meses)
Preguntas guía
- ¿A qué horas ocurren más accidentes?
- ¿Hay patrones nocturnos asociados a alcohol/drogas?
- ¿Hay días con más siniestros?
2. EDA Geográfico
Columnas: coordenada_x_utm, coordenada_y_utm, distrito, cod_distrito, localizacion
Qué puedes analizar
- Mapa de calor de accidentes (heatmap)
- Accidentes por distrito
- Puntos negros (localizaciones repetidas)
- Comparación entre distritos
Preguntas guía
- ¿Qué distritos concentran más accidentes?
- ¿Qué intersecciones aparecen repetidas?
3. EDA por Tipo de Accidente
Columnas: tipo_accidente, estado_meteorológico
Qué puedes analizar
- Frecuencia de cada tipo de accidente
- Relación entre clima y tipo de accidente
- Accidentes en días despejados vs lluvia
Preguntas guía
- ¿Qué tipo de accidente es más común?
- ¿Influye el clima en la severidad?
4. EDA por Vehículos y Personas
Columnas: tipo_vehiculo, tipo_persona, rango_edad, sexo
Qué puedes analizar
- Vehículos más involucrados
- Distribución por edad
- Distribución por sexo
- Conductores vs pasajeros vs peatones
- Accidentes de motos vs coches
Preguntas guía
- ¿Qué vehículos tienen más siniestros?
- ¿Qué grupos de edad aparecen más?
- ¿Hay diferencias entre hombres y mujeres?
5. EDA de Lesividad (muy importante)
Columnas: cod_lesividad, lesividad
Qué puedes analizar
- Severidad de los accidentes
- Relación entre tipo de accidente y lesividad
- Relación entre tipo de vehículo y lesividad
- Lesividad por edad y sexo
Preguntas guía
- ¿Qué tipo de accidente genera más heridos graves?
- ¿Qué edades presentan mayor lesividad?
6. EDA de Alcohol y Drogas
Columnas: positiva_alcohol, positiva_droga
Qué puedes analizar
- Accidentes con alcohol
- Accidentes con drogas
- Relación con hora del día
- Relación con tipo de accidente
- Relación con edad
Preguntas guía
- ¿A qué horas aparecen más positivos?
- ¿Qué tipo de accidente está más asociado a alcohol?
7. EDA de Duplicados por Expediente
Columna: num_expediente
Cada expediente puede tener varias filas (una por persona implicada).
Qué puedes analizar
- Número de personas por accidente
- Accidentes con múltiples implicados
- Relación entre número de implicados y severidad
8. EDA de Calidad del Dato
Columnas con NaN: cod_lesividad, lesividad, positiva_droga
Qué puedes analizar
- Porcentaje de valores faltantes
- Patrones de ausencia (¿ciertos distritos reportan menos?)
- Limpieza necesaria para análisis posteriores
9. EDA Avanzado (si quieres ir más lejos)
- Clustering de puntos de accidente (KMeans)
- Detección de outliers espaciales
- Mapas interactivos con Folium
- Series temporales si tienes varios meses


## Exploracion Inicial

Este Dataset es administrativo no estadistico, lo cual quiere decir que cada columna tiene un significado legal o procedimental, aqui un nulo no es un error, sino un estado administrativo, no puedes inventar datos porque alteras la realidad del registro.



La exploracion incial sirve para entender la estructura del Dataset anes de analizar cada variable, aqui no se tranfroma nada solo se observa.

## 2025

- Primero cargamos el dataset de acccidentalidad de madrid del año 2025:

In [ ]:
df_accidentes_2025 = pd.read_csv(
    "./data/2025_Accidentalidad.csv",
    sep=";",
    skip_blank_lines=True,
    on_bad_lines="skip",
    encoding="utf-8"
)



Ahora tambien quiero ver sus dimensiones, es decir cuantas filas y columnas tiene el Dataset

In [ ]:
df_accidentes_2025.shape

- Aqui observamos que tenemos 19 columnas y 41730 filas.

Ahora para el formato real de los datos y detectar valores a simple vista

In [ ]:
df_accidentes_2025.head()

Ahora para saber si cada columna esta en el tipo correcto y dectar problemas como por ejemplo numeros como textos, fechas mal cargadas, booleanos como strings,etc.


In [ ]:
df_accidentes_2025.info()

Aqui procedemos hacer una copia del dataset para dejar el original tal cual y poder ver diferencias a la hora de limpiar y demas.

In [ ]:
df_accidentes_2025_copy=df_accidentes_2025.copy()

Vamos a generar una tabla para ver los nulos y la cardinalidad de las diferentes variables que nos ayudaran a tomar decisiones con cada variable.
- La cardinalidad de las diferentes variables para decidir cuales son categoricas o no. Se calcula de la siguiente manera: 
$$Cardinalidad(<Columna/Característica>) = \dfrac{NúmerodeValoresUnicos(<Columna/Característica>)}{NúmeroTotalDeRegistros}* 100\%$$

In [ ]:
# Número de nulos 
nulos = df_accidentes_2025_copy.isna().sum() 
# Porcentaje de nulos 
nulos_pct = (nulos / len(df_accidentes_2025_copy) * 100).round(2) 
# Cardinalidad 
card = df_accidentes_2025_copy.nunique() 
# Porcentaje de cardinalidad 
card_pct = (card / len(df_accidentes_2025_copy) * 100).round(2) 
# Tipo de dato 
tipos = df_accidentes_2025_copy.dtypes 
#Construcción de la tabla 
tabla_eda = pd.DataFrame({ 
    "nulos": nulos,
    "% nulos": nulos_pct,
    "cardinalidad": card, 
    "% cardinalidad": card_pct,
    "tipo": tipos }) 
# Ordenar por cardinalidad (o por nulos si prefieres) 
tabla_eda = tabla_eda.sort_values("cardinalidad", ascending=False) 
tabla_eda

-Calculo la caridnalidad de la varibale antes de tratar los nulos porque sino estaria alternando artificalmente la cardinalidad de las variables.

-A partir de aqui tomo decisiones para modificar las variables al tipo que creemos mas acertado.

| Variable          | Tipo original | Tipo final             | Descripción                        | Importancia | Notas | 
|-------------------|---------------|------------------------|------------------------------------|-------------|-------| 
|num_expediente     | object        | string                 | Identificador único del expediente | Baja        | Cardinalidad muy alta, no se analiza, solo ID. | 
|fecha | object | datetime | Fecha del accidente | Alta | Necesaria para análisis temporal. | 
| hora | object | datetime.time | Hora del accidente | Alta | Pandas no tiene dtype específico; se mantiene como time. | 
| fecha_hora | derivada | datetime | Combinación de fecha y hora | Alta | Útil para análisis temporal preciso. | 
| localizacion | object | string | Dirección o punto exacto del accidente | Media | Cardinalidad muy alta; no convertir a category. | 
| numero | object | string | Número de la vía | Baja | No es numérico real, parte de la dirección. | 
| cod_distrito | int64 | category | Código numérico del distrito | Media | Categórica nominal; cardinalidad baja. | 
| distrito | object | category | Nombre del distrito | Alta | Variable clave para análisis espacial. | 
| tipo_accidente | object | category | Tipo de accidente | Alta | Cardinalidad baja; categórica nominal. | 
| estado_meteorológico | object | category | Condición meteorológica | Media | Imputada con 'No consta', categórica nominal. | 
| tipo_vehiculo | object | category | Tipo de vehículo implicado | Alta | Cardinalidad media, categórica nominal. | 
| tipo_persona | object | category | Rol de la persona (conductor, peatón...) | Alta | Categórica nominal. | 
| rango_edad | object | category (ordinal) | Rango de edad | Alta | Tiene orden natural, convertir a ordinal. | 
| sexo | object | category | Sexo de la persona | Media | Categórica nominal. | 
| cod_lesividad | float64 | category (ordinal) | Código de gravedad | Alta | Representa niveles de gravedad, imputado. | 
| lesividad | object | category (ordinal) | Gravedad del accidente | Alta | Variable clave, orden natural. | 
| coordenada_x_utm | float64 | float64 | Coordenada UTM X | Alta | Numérica continua, útil para mapas. | 
| coordenada_y_utm | float64 | float64 | Coordenada UTM Y | Alta | Numérica continua, útil para mapas. | 
| positiva_alcohol | object | category | Resultado alcohol | Media | Binaria + 'No consta'. | 
| positiva_droga | int64 | category | Resultado drogas | Baja | Cardinalidad 1 → variable inútil, se puede eliminar. |

In [ ]:
df_accidentes_2025_copy["fecha"] = pd.to_datetime(df_accidentes_2025_copy["fecha"], errors="coerce", dayfirst=True) 
df_accidentes_2025_copy["hora"] = pd.to_datetime(df_accidentes_2025_copy["hora"], format="%H:%M:%S", errors="coerce").dt.time 
df_accidentes_2025_copy["fecha_hora"] = df_accidentes_2025_copy.apply( 
                            lambda row: pd.Timestamp.combine(row["fecha"], row["hora"])
                            if pd.notnull(row["fecha"]) and pd.notnull(row["hora"])
                            else pd.NaT,
                            axis=1 )

cols_string = ["num_expediente", "localizacion", "numero"] 
for col in cols_string: 
    df_accidentes_2025_copy[col] = df_accidentes_2025_copy[col].astype(str).str.lower().str.strip().str.replace(r"\s+", " ", regex=True)
    

- Ahora vamos a tratar los nulos 

In [ ]:
#Estado meteorlogico
df_accidentes_2025_copy['estado_meteorológico'] = df_accidentes_2025_copy['estado_meteorológico'].fillna('No consta') 
# 2. TIPO VEHÍCULO 
mask_no_aplica = df_accidentes_2025_copy['tipo_persona'].str.lower().isin(['peatón', 'testigo']) 
# Peatón y testigo → No aplica 
df_accidentes_2025_copy.loc[ mask_no_aplica & df_accidentes_2025_copy['tipo_vehiculo'].isna(), 'tipo_vehiculo' ] = 'No aplica' 
# El resto de nulos → No consta 
df_accidentes_2025_copy['tipo_vehiculo'] = df_accidentes_2025_copy['tipo_vehiculo'].fillna('No consta') 
# 3. LESIVIDAD (ORDINAL) 
df_accidentes_2025_copy['cod_lesividad'] = df_accidentes_2025_copy['cod_lesividad'].fillna(14) 
df_accidentes_2025_copy['lesividad'] = df_accidentes_2025_copy['lesividad'].fillna('Sin asistencia sanitaria') 
# 4. ALCOHOL Y DROGAS 
df_accidentes_2025_copy['positiva_alcohol'] = df_accidentes_2025_copy['positiva_alcohol'].fillna('No consta') 
df_accidentes_2025_copy['positiva_droga'] = df_accidentes_2025_copy['positiva_droga'].fillna('No consta') 
#5. NUMERO 
df_accidentes_2025_copy['numero'] = df_accidentes_2025_copy['numero'].fillna('No aplica') 



In [ ]:
# 6. CONVERTIR A CATEGORY (DESPUÉS DE IMPUTAR) 
cols_category = [ 'estado_meteorológico', 
                 'tipo_vehiculo', 
                 'tipo_persona', 
                 'tipo_accidente', 
                 'distrito', 
                 'cod_distrito', 
                 'rango_edad', 
                 'sexo', 
                 'cod_lesividad', 
                 'lesividad', 
                 'positiva_alcohol', 
                 'positiva_droga' 
                 ] 
for col in cols_category: 
    df_accidentes_2025_copy[col] = df_accidentes_2025_copy[col].astype('category')

In [ ]:
df_accidentes_2025_copy.info()

In [ ]:
df_accidentes_2025_copy.head()

## 2024

- Primero cargamos el dataset de acccidentalidad de madrid del año 2024:

In [ ]:
df_accidentes_2024 = pd.read_csv(
    "./data/2024_Accidentalidad.csv",
    sep=";",
    skip_blank_lines=True,
    on_bad_lines="skip",
    encoding="utf-8"
)



Ahora tambien quiero ver sus dimensiones, es decir cuantas filas y columnas tiene el Dataset

In [ ]:
df_accidentes_2024.shape

-Aqui observamos que tiene 19 columnas igual que 2025 y tiene 49340 filas

Ahora para el formato real de los datos y detectar valores a simple vista

In [ ]:
df_accidentes_2024.head()

In [ ]:
df_accidentes_2024.info()

Aqui procedemos hacer una copia del dataset para dejar el original tal cual y poder ver diferencias a la hora de limpiar y demas.

In [ ]:
df_accidentes_2024_copy=df_accidentes_2024.copy()

Vamos a generar una tabla para ver los nulos y la cardinalidad de las diferentes variables que nos ayudaran a tomar decisiones con cada variable.
- La cardinalidad de las diferentes variables para decidir cuales son categoricas o no. Se calcula de la siguiente manera: 
$$Cardinalidad(<Columna/Característica>) = \dfrac{NúmerodeValoresUnicos(<Columna/Característica>)}{NúmeroTotalDeRegistros}* 100\%$$

In [ ]:
# Número de nulos 
nulos = df_accidentes_2024_copy.isna().sum() 
# Porcentaje de nulos 
nulos_pct = (nulos / len(df_accidentes_2024_copy) * 100).round(2) 
# Cardinalidad 
card = df_accidentes_2024_copy.nunique() 
# Porcentaje de cardinalidad 
card_pct = (card / len(df_accidentes_2024_copy) * 100).round(2) 
# Tipo de dato 
tipos = df_accidentes_2024_copy.dtypes 
#Construcción de la tabla 
tabla_eda = pd.DataFrame({ 
    "nulos": nulos,
    "% nulos": nulos_pct,
    "cardinalidad": card, 
    "% cardinalidad": card_pct,
    "tipo": tipos }) 
# Ordenar por cardinalidad (o por nulos si prefieres) 
tabla_eda = tabla_eda.sort_values("cardinalidad", ascending=False) 
tabla_eda

In [ ]:
df_accidentes_2024_copy["fecha"] = pd.to_datetime(df_accidentes_2024_copy["fecha"], errors="coerce", dayfirst=True) 
df_accidentes_2024_copy["hora"] = pd.to_datetime(df_accidentes_2024_copy["hora"], format="%H:%M:%S", errors="coerce").dt.time 
df_accidentes_2024_copy["fecha_hora"] = df_accidentes_2024_copy.apply( 
                            lambda row: pd.Timestamp.combine(row["fecha"], row["hora"])
                            if pd.notnull(row["fecha"]) and pd.notnull(row["hora"])
                            else pd.NaT,
                            axis=1 )

cols_string = ["num_expediente", "localizacion", "numero"] 
for col in cols_string: 
    df_accidentes_2024_copy[col] = df_accidentes_2024_copy[col].astype(str).str.lower().str.strip().str.replace(r"\s+", " ", regex=True)

In [ ]:

# 1. ESTADO METEOROLÓGICO

if 'estado_meteorológico' in df_accidentes_2024_copy.columns:
    df_accidentes_2024_copy['estado_meteorológico'] = df_accidentes_2024_copy['estado_meteorológico'].fillna('No consta')

# 2. TIPO VEHÍCULO

if 'tipo_vehiculo' in df_accidentes_2024_copy.columns and 'tipo_persona' in df_accidentes_2024_copy.columns:

    # Peatón y testigo → No aplica
    mask_no_aplica = df_accidentes_2024_copy['tipo_persona'].str.lower().isin(['peatón', 'testigo'])
    df_accidentes_2024_copy.loc[
        mask_no_aplica & df_accidentes_2024_copy['tipo_vehiculo'].isna(),
        'tipo_vehiculo'
    ] = 'No aplica'

    # El resto de nulos → No consta
    df_accidentes_2024_copy['tipo_vehiculo'] = df_accidentes_2024_copy['tipo_vehiculo'].fillna('No consta')

# 3. LESIVIDAD (ORDINAL)

if 'cod_lesividad' in df_accidentes_2024_copy.columns:
    df_accidentes_2024_copy['cod_lesividad'] = df_accidentes_2024_copy['cod_lesividad'].fillna(14)

if 'lesividad' in df_accidentes_2024_copy.columns:
    df_accidentes_2024_copy['lesividad'] = df_accidentes_2024_copy['lesividad'].fillna('Sin asistencia sanitaria')

# 4. ALCOHOL Y DROGAS

if 'positiva_alcohol' in df_accidentes_2024_copy.columns:
    df_accidentes_2024_copy['positiva_alcohol'] = df_accidentes_2024_copy['positiva_alcohol'].fillna('No consta')

if 'positiva_droga' in df_accidentes_2024_copy.columns:
    df_accidentes_2024_copy['positiva_droga'] = df_accidentes_2024_copy['positiva_droga'].fillna('No consta')

# 5. NUMERO

if 'numero' in df_accidentes_2024_copy.columns:
    df_accidentes_2024_copy['numero'] = df_accidentes_2024_copy['numero'].fillna('No aplica')



In [ ]:
# 6. CONVERTIR A CATEGORY (DESPUÉS DE IMPUTAR) 
cols_category = [ 'estado_meteorológico', 
                 'tipo_vehiculo', 
                 'tipo_persona', 
                 'tipo_accidente', 
                 'distrito', 
                 'cod_distrito', 
                 'rango_edad', 
                 'sexo', 
                 'cod_lesividad', 
                 'lesividad', 
                 'positiva_alcohol', 
                 'positiva_droga' 
                 ] 
for col in cols_category: 
    df_accidentes_2024_copy[col] = df_accidentes_2024_copy[col].astype('category')

In [ ]:
df_accidentes_2025_copy.info()

In [ ]:
df_accidentes_2024_copy=df_accidentes_2024.copy()

#La colmna de la fecha la paso a datatime:

df_accidentes_2024_copy["fecha"] = pd.to_datetime(df_accidentes_2024_copy["fecha"], errors="coerce", dayfirst=True)

#Aqui convierto la columna localizacion a string:

df_accidentes_2024_copy["localizacion"] = df_accidentes_2024_copy["localizacion"].astype(str)

#La coumna localizacion voy a pasarla a minuscula toda por si luego hacemos recuento de cuentas veces se repiten:

df_accidentes_2024_copy["localizacion"] = (
    df_accidentes_2024_copy["localizacion"]
    .str.lower()                       # minúsculas
    .str.strip()                       # quitar espacios extremos
    .str.replace(r"\s+", " ", regex=True)  # espacios múltiples → uno solo
)
df_accidentes_2024_copy["distrito"] = df_accidentes_2024_copy["distrito"].astype(str)

#La coumna localizacion voy a pasarla a minuscula toda por si luego hacemos recuento de cuentas veces se repiten:

df_accidentes_2024_copy["distrito"] = (
    df_accidentes_2024_copy["distrito"]
    .str.lower()                       # minúsculas
    .str.strip()                       # quitar espacios extremos
    .str.replace(r"\s+", " ", regex=True)  # espacios múltiples → uno solo
)
df_accidentes_2024_copy

In [ ]:
df_accidentes_2024_copy['hora'] = pd.to_datetime(df_accidentes_2024_copy['hora'], format='%H:%M:%S', errors='coerce').dt.time
#df_accidentes_copy['hora'].head() df_accidentes_copy['hora'].dtype # seguirá siendo object (datetime.time)

df_accidentes_2024_copy['fecha_hora'] = df_accidentes_2024_copy.apply( lambda row: pd.Timestamp.combine(row['fecha'], row['hora']) if pd.notnull(row['fecha']) and pd.notnull(row['hora']) else pd.NaT, axis=1 )


df_accidentes_2024_copy['numero'] = pd.to_numeric(df_accidentes_2024_copy['numero'], errors='coerce') 
df_accidentes_2024_copy['positiva_alcohol'] = df_accidentes_2024_copy['positiva_alcohol'].map({'S':1, 'N':0}) 
df_accidentes_2024_copy['positiva_droga'] = df_accidentes_2024_copy['positiva_droga'].fillna(0).astype(int)

df_accidentes_2024_copy

In [ ]:
df_accidentes_2024_copy.info()

In [ ]:
# 1. estado_meteorológico 

df_accidentes_2024_copy['estado_meteorológico'] = df_accidentes_2024_copy['estado_meteorológico'].fillna('No consta') 

# # 2. tipo_vehiculo

mask_no_aplica = df_accidentes_2024_copy['tipo_persona'].str.lower().isin(['peatón', 'testigo']) 

df_accidentes_2024_copy.loc[mask_no_aplica & df_accidentes_2024_copy['tipo_vehiculo'].isna(), 'tipo_vehiculo'] = 'No aplica' 


# Rellenar el resto de nulos con "No consta" 

df_accidentes_2024_copy['tipo_vehiculo'] = df_accidentes_2024_copy['tipo_vehiculo'].fillna('No consta')

# 3. Lesividad 

df_accidentes_2024_copy['cod_lesividad'] = df_accidentes_2024_copy['cod_lesividad'].fillna(14) 

df_accidentes_2024_copy['lesividad'] = df_accidentes_2024_copy['lesividad'].fillna('Sin asistencia sanitaria') 

# 4. Alcohol y drogas 

df_accidentes_2024_copy['positiva_alcohol'] = df_accidentes_2024_copy['positiva_alcohol'].fillna('No consta') 

df_accidentes_2024_copy['positiva_droga'] = df_accidentes_2024_copy['positiva_droga'].fillna('No consta')

In [ ]:
df_accidentes_2024_copy.info()

- Tratamiento 2023

In [ ]:
df_accidentes_2023 = pd.read_csv(
    "./data/2023_Accidentalidad.csv",
    sep=";",
    skip_blank_lines=True,
    on_bad_lines="skip",
    encoding="utf-8"
)
df_accidentes_2023

In [ ]:
df_accidentes_2023.info()

In [ ]:
df_accidentes_2023_copy=df_accidentes_2023.copy()

#La colmna de la fecha la paso a datatime:

df_accidentes_2023_copy["fecha"] = pd.to_datetime(df_accidentes_2023_copy["fecha"], errors="coerce", dayfirst=True)

#Aqui convierto la columna localizacion a string:

df_accidentes_2023_copy["localizacion"] = df_accidentes_2023_copy["localizacion"].astype(str)

#La coumna localizacion voy a pasarla a minuscula toda por si luego hacemos recuento de cuentas veces se repiten:

df_accidentes_2023_copy["localizacion"] = (
    df_accidentes_2023_copy["localizacion"]
    .str.lower()                       # minúsculas
    .str.strip()                       # quitar espacios extremos
    .str.replace(r"\s+", " ", regex=True)  # espacios múltiples → uno solo
)
df_accidentes_2023_copy["distrito"] = df_accidentes_2023_copy["distrito"].astype(str)

#La coumna localizacion voy a pasarla a minuscula toda por si luego hacemos recuento de cuentas veces se repiten:

df_accidentes_2023_copy["distrito"] = (
    df_accidentes_2023_copy["distrito"]
    .str.lower()                       # minúsculas
    .str.strip()                       # quitar espacios extremos
    .str.replace(r"\s+", " ", regex=True)  # espacios múltiples → uno solo
)
df_accidentes_2023_copy

In [ ]:
df_accidentes_2023_copy['hora'] = pd.to_datetime(df_accidentes_2023_copy['hora'], format='%H:%M:%S', errors='coerce').dt.time
#df_accidentes_copy['hora'].head() df_accidentes_copy['hora'].dtype # seguirá siendo object (datetime.time)

df_accidentes_2023_copy['fecha_hora'] = df_accidentes_2023_copy.apply( lambda row: pd.Timestamp.combine(row['fecha'], row['hora']) if pd.notnull(row['fecha']) and pd.notnull(row['hora']) else pd.NaT, axis=1 )


df_accidentes_2023_copy['numero'] = pd.to_numeric(df_accidentes_2023_copy['numero'], errors='coerce') 
df_accidentes_2023_copy['positiva_alcohol'] = df_accidentes_2023_copy['positiva_alcohol'].map({'S':1, 'N':0}) 
df_accidentes_2023_copy['positiva_droga'] = df_accidentes_2023_copy['positiva_droga'].fillna(0).astype(int)

df_accidentes_2023_copy

In [ ]:
df_accidentes_2023_copy.info()

In [ ]:
# 1. estado_meteorológico 

df_accidentes_2023_copy['estado_meteorológico'] = df_accidentes_2023_copy['estado_meteorológico'].fillna('No consta') 

# # 2. tipo_vehiculo

mask_no_aplica = df_accidentes_2023_copy['tipo_persona'].str.lower().isin(['peatón', 'testigo']) 

df_accidentes_2023_copy.loc[mask_no_aplica & df_accidentes_2023_copy['tipo_vehiculo'].isna(), 'tipo_vehiculo'] = 'No aplica' 


# Rellenar el resto de nulos con "No consta" 

df_accidentes_2023_copy['tipo_vehiculo'] = df_accidentes_2023_copy['tipo_vehiculo'].fillna('No consta')

# 3. Lesividad 

df_accidentes_2023_copy['cod_lesividad'] = df_accidentes_2023_copy['cod_lesividad'].fillna(14) 

df_accidentes_2023_copy['lesividad'] = df_accidentes_2023_copy['lesividad'].fillna('Sin asistencia sanitaria') 

# 4. Alcohol y drogas 

df_accidentes_2023_copy['positiva_alcohol'] = df_accidentes_2023_copy['positiva_alcohol'].fillna('No consta') 

df_accidentes_2023_copy['positiva_droga'] = df_accidentes_2023_copy['positiva_droga'].fillna('No consta')

In [ ]:
df_accidentes_2023_copy.info()

- Tratamiento 2022

In [ ]:
df_accidentes_2022 = pd.read_csv(
    "./data/2022_Accidentalidad.csv",
    sep=";",
    skip_blank_lines=True,
    on_bad_lines="skip",
    encoding="utf-8"
)
df_accidentes_2022

In [ ]:
df_accidentes_2022_copy=df_accidentes_2022.copy()

#La colmna de la fecha la paso a datatime:

df_accidentes_2022_copy["fecha"] = pd.to_datetime(df_accidentes_2022_copy["fecha"], errors="coerce", dayfirst=True)

#Aqui convierto la columna localizacion a string:

df_accidentes_2022_copy["localizacion"] = df_accidentes_2022_copy["localizacion"].astype(str)

#La coumna localizacion voy a pasarla a minuscula toda por si luego hacemos recuento de cuentas veces se repiten:

df_accidentes_2022_copy["localizacion"] = (
    df_accidentes_2022_copy["localizacion"]
    .str.lower()                       # minúsculas
    .str.strip()                       # quitar espacios extremos
    .str.replace(r"\s+", " ", regex=True)  # espacios múltiples → uno solo
)
df_accidentes_2022_copy["distrito"] = df_accidentes_2022_copy["distrito"].astype(str)

#La coumna localizacion voy a pasarla a minuscula toda por si luego hacemos recuento de cuentas veces se repiten:

df_accidentes_2022_copy["distrito"] = (
    df_accidentes_2022_copy["distrito"]
    .str.lower()                       # minúsculas
    .str.strip()                       # quitar espacios extremos
    .str.replace(r"\s+", " ", regex=True)  # espacios múltiples → uno solo
)
df_accidentes_2022_copy

In [ ]:
df_accidentes_2022_copy['hora'] = pd.to_datetime(df_accidentes_2022_copy['hora'], format='%H:%M:%S', errors='coerce').dt.time
#df_accidentes_copy['hora'].head() df_accidentes_copy['hora'].dtype # seguirá siendo object (datetime.time)

df_accidentes_2022_copy['fecha_hora'] = df_accidentes_2022_copy.apply( lambda row: pd.Timestamp.combine(row['fecha'], row['hora']) if pd.notnull(row['fecha']) and pd.notnull(row['hora']) else pd.NaT, axis=1 )


df_accidentes_2022_copy['numero'] = pd.to_numeric(df_accidentes_2022_copy['numero'], errors='coerce') 
df_accidentes_2022_copy['positiva_alcohol'] = df_accidentes_2022_copy['positiva_alcohol'].map({'S':1, 'N':0}) 
df_accidentes_2022_copy['positiva_droga'] = df_accidentes_2022_copy['positiva_droga'].fillna(0).astype(int)

df_accidentes_2022_copy

In [ ]:
df_accidentes_2022_copy.info()

In [ ]:
# 1. estado_meteorológico 

df_accidentes_2022_copy['estado_meteorológico'] = df_accidentes_2022_copy['estado_meteorológico'].fillna('No consta') 

# # 2. tipo_vehiculo

mask_no_aplica = df_accidentes_2022_copy['tipo_persona'].str.lower().isin(['peatón', 'testigo']) 

df_accidentes_2022_copy.loc[mask_no_aplica & df_accidentes_2022_copy['tipo_vehiculo'].isna(), 'tipo_vehiculo'] = 'No aplica' 


# Rellenar el resto de nulos con "No consta" 

df_accidentes_2022_copy['tipo_vehiculo'] = df_accidentes_2022_copy['tipo_vehiculo'].fillna('No consta')

# 3. Lesividad 

df_accidentes_2022_copy['cod_lesividad'] = df_accidentes_2022_copy['cod_lesividad'].fillna(14) 

df_accidentes_2022_copy['lesividad'] = df_accidentes_2022_copy['lesividad'].fillna('Sin asistencia sanitaria') 

# 4. Alcohol y drogas 

df_accidentes_2022_copy['positiva_alcohol'] = df_accidentes_2022_copy['positiva_alcohol'].fillna('No consta') 

df_accidentes_2022_copy['positiva_droga'] = df_accidentes_2022_copy['positiva_droga'].fillna('No consta')

In [ ]:
df_accidentes_2022_copy.info()

- Tratamiento 2021

In [ ]:
df_accidentes_2021 = pd.read_csv(
    "./data/2021_Accidentalidad.csv",
    sep=";",
    skip_blank_lines=True,
    on_bad_lines="skip",
    encoding="utf-8"
)
df_accidentes_2021

In [ ]:
df_accidentes_2021_copy=df_accidentes_2021.copy()

#La colmna de la fecha la paso a datatime:

df_accidentes_2021_copy["fecha"] = pd.to_datetime(df_accidentes_2021_copy["fecha"], errors="coerce", dayfirst=True)

#Aqui convierto la columna localizacion a string:

df_accidentes_2021_copy["localizacion"] = df_accidentes_2021_copy["localizacion"].astype(str)

#La coumna localizacion voy a pasarla a minuscula toda por si luego hacemos recuento de cuentas veces se repiten:

df_accidentes_2021_copy["localizacion"] = (
    df_accidentes_2021_copy["localizacion"]
    .str.lower()                       # minúsculas
    .str.strip()                       # quitar espacios extremos
    .str.replace(r"\s+", " ", regex=True)  # espacios múltiples → uno solo
)
df_accidentes_2021_copy["distrito"] = df_accidentes_2021_copy["distrito"].astype(str)

#La coumna localizacion voy a pasarla a minuscula toda por si luego hacemos recuento de cuentas veces se repiten:

df_accidentes_2021_copy["distrito"] = (
    df_accidentes_2021_copy["distrito"]
    .str.lower()                       # minúsculas
    .str.strip()                       # quitar espacios extremos
    .str.replace(r"\s+", " ", regex=True)  # espacios múltiples → uno solo
)
df_accidentes_2021_copy

In [ ]:
df_accidentes_2021_copy['hora'] = pd.to_datetime(df_accidentes_2021_copy['hora'], format='%H:%M:%S', errors='coerce').dt.time
#df_accidentes_copy['hora'].head() df_accidentes_copy['hora'].dtype # seguirá siendo object (datetime.time)

df_accidentes_2021_copy['fecha_hora'] = df_accidentes_2021_copy.apply( lambda row: pd.Timestamp.combine(row['fecha'], row['hora']) if pd.notnull(row['fecha']) and pd.notnull(row['hora']) else pd.NaT, axis=1 )


df_accidentes_2021_copy['numero'] = pd.to_numeric(df_accidentes_2021_copy['numero'], errors='coerce') 
df_accidentes_2021_copy['positiva_alcohol'] = df_accidentes_2021_copy['positiva_alcohol'].map({'S':1, 'N':0}) 
df_accidentes_2021_copy['positiva_droga'] = df_accidentes_2021_copy['positiva_droga'].fillna(0).astype(int)

df_accidentes_2021_copy

In [ ]:
# 1. estado_meteorológico 

df_accidentes_2021_copy['estado_meteorológico'] = df_accidentes_2021_copy['estado_meteorológico'].fillna('No consta') 

# # 2. tipo_vehiculo

mask_no_aplica = df_accidentes_2021_copy['tipo_persona'].str.lower().isin(['peatón', 'testigo']) 

df_accidentes_2021_copy.loc[mask_no_aplica & df_accidentes_2021_copy['tipo_vehiculo'].isna(), 'tipo_vehiculo'] = 'No aplica' 


# Rellenar el resto de nulos con "No consta" 

df_accidentes_2021_copy['tipo_vehiculo'] = df_accidentes_2021_copy['tipo_vehiculo'].fillna('No consta')

# 3. Lesividad 

df_accidentes_2021_copy['cod_lesividad'] = df_accidentes_2021_copy['cod_lesividad'].fillna(14) 

df_accidentes_2021_copy['lesividad'] = df_accidentes_2021_copy['lesividad'].fillna('Sin asistencia sanitaria') 

# 4. Alcohol y drogas 

df_accidentes_2021_copy['positiva_alcohol'] = df_accidentes_2021_copy['positiva_alcohol'].fillna('No consta') 

df_accidentes_2021_copy['positiva_droga'] = df_accidentes_2021_copy['positiva_droga'].fillna('No consta')

In [ ]:
df_accidentes_2021_copy.info()

- Tratamiento 2020

In [ ]:
df_accidentes_2020 = pd.read_csv(
    "./data/2020_Accidentalidad.csv",
    sep=";",
    skip_blank_lines=True,
    on_bad_lines="skip",
    encoding="utf-8"
)
df_accidentes_2020

In [ ]:
df_accidentes_2020.info()

In [ ]:
df_accidentes_2020_copy=df_accidentes_2020.copy()

#La colmna de la fecha la paso a datatime:

df_accidentes_2020_copy["fecha"] = pd.to_datetime(df_accidentes_2020_copy["fecha"], errors="coerce", dayfirst=True)

#Aqui convierto la columna localizacion a string:

df_accidentes_2020_copy["localizacion"] = df_accidentes_2020_copy["localizacion"].astype(str)

#La coumna localizacion voy a pasarla a minuscula toda por si luego hacemos recuento de cuentas veces se repiten:

df_accidentes_2020_copy["localizacion"] = (
    df_accidentes_2020_copy["localizacion"]
    .str.lower()                       # minúsculas
    .str.strip()                       # quitar espacios extremos
    .str.replace(r"\s+", " ", regex=True)  # espacios múltiples → uno solo
)
df_accidentes_2020_copy["distrito"] = df_accidentes_2020_copy["distrito"].astype(str)

#La coumna localizacion voy a pasarla a minuscula toda por si luego hacemos recuento de cuentas veces se repiten:

df_accidentes_2020_copy["distrito"] = (
    df_accidentes_2020_copy["distrito"]
    .str.lower()                       # minúsculas
    .str.strip()                       # quitar espacios extremos
    .str.replace(r"\s+", " ", regex=True)  # espacios múltiples → uno solo
)
df_accidentes_2020_copy

In [ ]:
df_accidentes_2020_copy['hora'] = pd.to_datetime(df_accidentes_2020_copy['hora'], format='%H:%M:%S', errors='coerce').dt.time
#df_accidentes_copy['hora'].head() df_accidentes_copy['hora'].dtype # seguirá siendo object (datetime.time)

df_accidentes_2020_copy['fecha_hora'] = df_accidentes_2020_copy.apply( lambda row: pd.Timestamp.combine(row['fecha'], row['hora']) if pd.notnull(row['fecha']) and pd.notnull(row['hora']) else pd.NaT, axis=1 )


df_accidentes_2020_copy['numero'] = pd.to_numeric(df_accidentes_2020_copy['numero'], errors='coerce') 
df_accidentes_2020_copy['positiva_alcohol'] = df_accidentes_2020_copy['positiva_alcohol'].map({'S':1, 'N':0}) 
df_accidentes_2020_copy['positiva_droga'] = df_accidentes_2020_copy['positiva_droga'].fillna(0).astype(int)


In [ ]:
# 1. estado_meteorológico 

df_accidentes_2020_copy['estado_meteorológico'] = df_accidentes_2020_copy['estado_meteorológico'].fillna('No consta') 

# # 2. tipo_vehiculo

mask_no_aplica = df_accidentes_2020_copy['tipo_persona'].str.lower().isin(['peatón', 'testigo']) 

df_accidentes_2020_copy.loc[mask_no_aplica & df_accidentes_2020_copy['tipo_vehiculo'].isna(), 'tipo_vehiculo'] = 'No aplica' 


# Rellenar el resto de nulos con "No consta" 

df_accidentes_2020_copy['tipo_vehiculo'] = df_accidentes_2020_copy['tipo_vehiculo'].fillna('No consta')

# 3. Lesividad 

df_accidentes_2020_copy['cod_lesividad'] = df_accidentes_2020_copy['cod_lesividad'].fillna(14) 

df_accidentes_2020_copy['lesividad'] = df_accidentes_2020_copy['lesividad'].fillna('Sin asistencia sanitaria') 

# 4. Alcohol y drogas 

df_accidentes_2020_copy['positiva_alcohol'] = df_accidentes_2020_copy['positiva_alcohol'].fillna('No consta') 

df_accidentes_2020_copy['positiva_droga'] = df_accidentes_2020_copy['positiva_droga'].fillna('No consta')

In [ ]:
df_accidentes_2020_copy.info()

- cosas varias

In [ ]:
df_accidentes_copy["lesivilidad"].unique()

In [ ]:
type(df_accidentes_copy['hora'].iloc[0])

- Variables temporales útiles

In [ ]:
df_accidentes_copy['anio'] = df_accidentes_copy['fecha'].dt.year 
df_accidentes_copy['mes'] = df_accidentes_copy['fecha'].dt.month 
df_accidentes_copy['dia_semana'] = df_accidentes_copy['fecha'].dt.day_name() 
df_accidentes_copy['hora_num'] = df_accidentes_copy['fecha_hora'].dt.hour

- Distribución por hora

In [ ]:
df_accidentes_copy['hora_num'].value_counts().sort_index().plot(kind='bar', figsize=(12,4))

- Franja horaria

In [ ]:
df_accidentes_copy['franja'] = pd.cut( df_accidentes_copy['hora_num'], bins=[-1,6,12,18,24], labels=['Madrugada','Mañana','Tarde','Noche'] )

- Accidentes por día de la semana

In [ ]:
df_accidentes_copy['dia_semana'].value_counts().plot(kind='bar')

### Otras Cosas 

| Columna/Variable    | Descripción |
| ----------------    | ----------- |
| num_expediente      | Indica si el pasajero sobrevivió (1) o no (0) |
| fecha               | Clase del pasaje (1ª, 2ª, 3ª) |
| hora                | Género del pasajero |
| localizacion        | Edad del pasajero |
| numero	          | Es el numero de familiares que viajan con la persona indicada en el manifiesto (lista de pasajeros) |
| cod_distrito        | Número de padres/hijos a bordo |
| distrito            | Tarifa pagada por el pasajero |
| tipo_accidente      | Es la incial de la letra de la ciudad donde embarcó la persona.(C = Cherbourg; Q = Queenstown; S = Southampton) |
| estado_meteorológico| Clase del pasaje como categoría (Primera, Segunda, Tercera) |
| tipo_vehiculo       | Categoría del pasajero (hombre, mujer, niño) |
| tipo_persona        | Booleano que indica si el pasajero es un hombre adulto |
| rango_edad          | Cubierta del barco donde se ubicaba el pasajero |
| sexo	              | Ciudad de embarque: Cherbourg; Queenstown; Southampton)  |
| cod_lesividad       | Indica si el pasajero está vivo ('yes') o muerto ('no') |
| lesividad           | Booleano que indica si el pasajero viajaba solo |
| coordenada_x_utm    | Booleano que indica si el pasajero viajaba solo |
| coordenada_y_utm    | Booleano que indica si el pasajero viajaba solo |
| positiva_alcohol    | Booleano que indica si el pasajero viajaba solo |
| positiva_droga      | Booleano que indica si el pasajero viajaba solo |

In [ ]:
df_accidentes[df_accidentes['num_expediente'].duplicated(keep=False)]

In [ ]:
len(df_accidentes['num_expediente'][df_accidentes['num_expediente'].duplicated()].unique())

In [ ]:
df_accidentes['num_expediente'].value_counts().reset_index(name='frecuencia').query("frecuencia < 10")

In [ ]:
df_accidentes['num_expediente'].dtype

In [ ]:
type(df_accidentes['num_expediente'].iloc[0])

In [ ]:
df_accidentes_copy["tipo_vehiculo"].unique()

In [ ]:
df_accidentes_copy["tipo_accidente"].unique()

| Nombre del campo                  | Descripción                                                                 | Tipo de variable | Importancia Inicial | Notas |
|-----------------------------------|----------------------------------------------------------------------------|------------------|---------------------|-------|
| customer                          | ID del cliente                                                             |                  |                     |       |
| state                             | Estados en US                                                              |                  |                     |       |
| customer_lifetime_value           | CLV es el valor económico del cliente para la empresa durante toda su relación |                  |                     |   En dolares    |
| response                          | Respuesta a campañas/llamadas (marketing-engagement)                       |                  |                     |       |
| coverage                          | Tipo de cobertura del cliente                                              |                  |                     |       |
| education                         | Nivel educativo del cliente                                                |                  |                     |       |
| effective_to_date                 | Fecha efectiva                                                             |                  |                     |       |
| employmentstatus                  | Estado laboral del cliente                                                 |                  |                     |       |
| gender                            | Género del cliente                                                         |                  |                     |       |
| income                            | Ingresos del cliente                                                       |                  |                     |       |
| location_code                     | Zona de residencia del cliente                                             |                  |                     |       |
| marital_status                    | Estado Civil del cliente                                                   |                  |                     |       |
| monthly_premium_auto              | Premium mensual                                                            |                  |                     |       |
| months_since_last_claim           | Última reclamación del cliente                                             |                  |                     |       |
| months_since_policy_inception     | Inicio de la póliza                                                        |                  |                     |       |
| number_of_open_complaints         | Reclamaciones abiertas                                                     |                  |                     |  Son partes, no quejas     |
| number_of_policies                | Número de pólizas                                                          |                  |                     |       |
| policy_type                       | Tipo de póliza                                                             |                  |                     |       |
| policy                            | Póliza                                                                     |                  |                     |       |
| renew_offer_type                  | Oferta de renovación                                                       |                  |                     |       |
| sales_channel                     | Canal de ventas (primer contacto compañía-cliente)                         |                  |                     |  Puede continuar en otro canal diferente     |
| total_claim_amount                | Monto de la reclamación                                                    |                  |                     |       |
| vehicle_class                     | Tipo de vehículo                                                           |                  |                     |       |
| vehicle_size                      | Tamaño del vehículo                                                        |                  |                     |       |


In [ ]:
pd.set_option('display.max_rows', None) 
pd.set_option('display.max_columns', None)

df_registros=df_accidentes['num_expediente'] .value_counts().reset_index(name='frecuencia').query("frecuencia < 23")
#df_registros